In [15]:
import ollama

### Initializing the model class:

In [16]:
class AIAgent:
    def __init__(self, model, system_prompt):
        self.model = model
        self.system_prompt = system_prompt

    def get_response(self, user_input):
        response = ollama.chat(model=self.model, messages=[
            {'role': 'system', 'content': self.system_prompt},
            {'role': 'user', 'content': user_input}
        ])
        return response['message']['content']

### Setting up agents:

In [17]:
# Code Analyzer Agent
code_analyzer = AIAgent(
    model="llama3",
    system_prompt="You are a code analyzer. Identify and list potential security problems or issues in the given code snippet. Focus on security vulnerabilities, bugs, and bad practices."
)

# Problem Reasoner Agent
problem_reasoner = AIAgent(
    model="llama3",
    system_prompt="You are a problem reasoner. Explain the reasons behind the code issues identified. Provide clear explanations for why these are problems and their potential impacts."
)

# Code Fixer Agent
code_fixer = AIAgent(
    model="llama3",
    system_prompt="You are a code fixer. Given a code snippet and identified issues, provide corrected code that addresses these problems. Explain your changes."
)

# Report Generator Agent
report_generator = AIAgent(
    model="llama3",
    system_prompt="You are a report generator. Compile the analysis, reasoning, and fixes into a comprehensive, well-structured report. Use markdown formatting for better readability."
)

### Prompting using llama3:

In [18]:
def analyze_and_fix_code(code_snippet):
    # Step 1: Analyze the code
    analysis = code_analyzer.get_response(f"Analyze this code:\n\n{code_snippet}")
    
    # Step 2: Reason about the problems
    reasoning = problem_reasoner.get_response(f"Explain these issues:\n\n{analysis}")
    
    # Step 3: Fix the code
    fixes = code_fixer.get_response(f"Fix this code based on these issues:\n\nCode:\n{code_snippet}\n\nIssues:\n{analysis}")
    
    # Step 4: Generate a report
    report = report_generator.get_response(f"""Generate a comprehensive report with the following sections:

    1. Original Code
    2. Identified Issues
    3. Problem Analysis
    4. Code Fixes
    5. Conclusion

    Use the following information:

    Original Code:
    {code_snippet}

    Identified Issues:
    {analysis}

    Problem Analysis:
    {reasoning}

    Code Fixes:
    {fixes}
    """)
    
    return report

# Example usage
code_to_analyze = """
let mut seen_nonces = BTreeSet::default();
let mut validated_txs = Vec::with_capacity(mint_config_txs.len());
for tx in mint_config_txs {
    // Ensure all nonces are unique.
    if !seen_nonces.insert(tx.prefix.nonce.clone()) {
        return Err(Error::FormBlock(format!(
            "Duplicate MintConfigTx nonce: {:?}",
            tx.prefix.nonce
        )));
    }
}
"""

ai_report = analyze_and_fix_code(code_to_analyze)
print(ai_report)

The report generator has compiled the analysis, reasoning, and fixes into a comprehensive, well-organized user-controlled input (tx.prefix.nonce). This could potentially lead to an attacker-controlled payload being injected into the error message, which could have unintended consequences downstream.

**Why it's a problem:** Injecting unvalidated user input into error messages can be a security vulnerability. An attacker could manipulate the error message to contain malicious payloads, such as code injection or data tampering.

The report generator has identified eight issues in the original code:

1. **Insecure Error Handling:** The original code injected attacker-controlled payload into the error message.
2. **Lack of Input Validation:** The code does not perform any validation on the `mint_ config_ txs` input.
3. **Mutable State:** The code maintains mutable state (`seen_nonces`, `validated_ txs`) that is not necessarily thread-safe.
4. **Lack of Documentation:** There is no document

### Evaluation against ground truth (tried to mimic human feedback using llama3):

In [20]:
def evaluate_against_ground_truth(ai_output, ground_truth):
    evaluator = AIAgent(
        model="llama3",
        system_prompt="""You are an expert code reviewer and evaluator. Your task is to compare the AI-generated analysis and fixes with the ground truth provided. 
        Evaluate the accuracy, completeness, and relevance of the AI's output. 
        Provide a score from 0 to 10 for each of the following categories:
        1. Issue Identification
        2. Problem Analysis
        3. Proposed Fixes
        4. Overall Accuracy
        
        Also, provide a brief explanation for each score and any discrepancies found."""
    )

    evaluation_prompt = f"""
    Compare the following AI-generated output with the provided ground truth:

    AI-generated output:
    {ai_output}

    Ground Truth:
    {ground_truth}

    Evaluate the AI output based on the criteria mentioned in the system prompt.
    """

    evaluation = evaluator.get_response(evaluation_prompt)
    return evaluation

# Example usage
code_to_analyze = """
let mut seen_nonces = BTreeSet::default();
let mut validated_txs = Vec::with_capacity(mint_config_txs.len());
for tx in mint_config_txs {
    // Ensure all nonces are unique.
    if !seen_nonces.insert(tx.prefix.nonce.clone()) {
        return Err(Error::FormBlock(format!(
            "Duplicate MintConfigTx nonce: {:?}",
            tx.prefix.nonce
        )));
    }
}
"""

ground_truth = """
Nonces are not stored per token
Severity: Low 
Difficulty: High

Mint and mint configuration transaction nonces are not distinguished by the tokens with
which they are associated. Malicious minters or governors could use this fact to conduct
denial-of-service attacks against other minters and governors.
The relevant code appears in figures 4.1 and 4.2. For each type of transaction, nonces are
inserted into a seen_nonces set without regard to the token indicated in the transaction.

let mut seen_nonces = BTreeSet::default();
let mut validated_txs = Vec::with_capacity(mint_config_txs.len());
for tx in mint_config_txs {
// Ensure all nonces are unique.
if !seen_nonces.insert(tx.prefix.nonce.clone()) {
return Err(Error::FormBlock(format!(
"Duplicate MintConfigTx nonce: {:?}",
tx.prefix.nonce
)));
}


let mut mint_txs = Vec::with_capacity(mint_txs_with_config.len());
let mut seen_nonces = BTreeSet::default();
for (mint_tx, mint_config_tx, mint_config) in mint_txs_with_config {
// The nonce should be unique.
if !seen_nonces.insert(mint_tx.prefix.nonce.clone()) {
return Err(Error::FormBlock(format!(
"Duplicate MintTx nonce: {:?}",
mint_tx.prefix.nonce
)));
}

"""

# Generate AI output
# ai_output = analyze_and_fix_code(code_to_analyze)

# Evaluate against ground truth
evaluation_result = evaluate_against_ground_truth(ai_report, ground_truth)

print("Evaluation Result:")
print(evaluation_result)

Evaluation Result:
**Issue Identification:** 8/10
The AI-generated output correctly identified eight issues with the original code, including Insecure Error Handling, Lack of Input Validation, Mutable State, Lack of Documentation, Unnecessary Allocation, Code Organization, Naming Conventions, and Memory Safety.

However, it did not identify the specific issue mentioned in the ground truth: "Nonces are not stored per token." This is a significant issue that affects the security of the code. Therefore, I deduct 2 points from the total score.

**Problem Analysis:** 7/10
The AI-generated output provides some analysis for each issue identified, but it does not go into as much detail as the ground truth. For example, the AI does not explain why storing nonces per token is important or how it affects security. Additionally, some of the issues mentioned in the AI-generated output are not directly related to the specific problem described in the ground truth.

**Proposed Fixes:** 9/10
The AI-ge